In [238]:
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()

Will the AND-1 riboswitch cleave itself when both of its OBS are bound?  
AND-1 will output TRUE

Will the OR-1 riboswitch cleave itself when neither of its OBS are bound?  
OR-1 will output FALSE

What behavior do we expect from the YES-1 riboswitch?  
Will output TRUE if OBS is bound and FALSE if not.

In [239]:
#Create riboswitch table, named riboswitch   

c.execute("""DROP TABLE riboswitch;""")
 
c.execute("""CREATE TABLE riboswitch 
(riboswitch_name TEXT, 
riboswitch_sequece TEXT, 
start_cleavage1 INT, 
end_cleavage1 INT, 
start_cleavage2 INT,
end_cleavage2 INT, 
start_OBS1 INT, 
end_OBS1 INT, 
start_OBS2 INT, 
end_OBS2 INT
);""")

#input data for constructing riboswitch table
riboswitch_data = [('YES-1', 'GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC'
                    , 16, 21, 49, 54, 26, 47, '', ''),
                   ('NOT-1','GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUGCGUCCUGUAUUCCACUGC', 
                   40, 43, 74, 77, 44, 66, '', ''),
                   ('AND-1','GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAUA', 
                   16, 23, 70, 77, 30, 45, 49, 64),
                   ('OR-1','GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC', 
                   16, 26, 67, 77, 27, 46, 47, 66)
                  ]

c.executemany('INSERT INTO riboswitch VALUES (?,?,?,?,?,?,?,?,?,?)', riboswitch_data)

conn.commit()

#Extract YES-1 riboswitch data from table and store into YES1
c.execute("SELECT * FROM riboswitch;")
YES1 = c.fetchall()[0]

#Extract NOT-1 riboswitch data from table and store into NOT1
c.execute("SELECT * FROM riboswitch;")
NOT1 = c.fetchall()[1]

#Extract AND-1 riboswitch data from table and store into AND1
c.execute("SELECT * FROM riboswitch;")
AND1 = c.fetchall()[2]

#Extract OR-1 riboswitch data from table and store into OR1
c.execute("SELECT * FROM riboswitch;")
OR1 = c.fetchall()[3]

print(YES1)
print(NOT1)
print(AND1)
print(OR1)

('YES-1', 'GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC', 16, 21, 49, 54, 26, 47, '', '')
('NOT-1', 'GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUGCGUCCUGUAUUCCACUGC', 40, 43, 74, 77, 44, 66, '', '')
('AND-1', 'GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAUA', 16, 23, 70, 77, 30, 45, 49, 64)
('OR-1', 'GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC', 16, 26, 67, 77, 27, 46, 47, 66)


In [240]:
import subprocess

#RNAfold and plot for all riboswitches, without contraints

#Input all four riboswitch sequences
seq_all = """>YES1
""" + YES1[1] + """
>NOT1
""" + NOT1[1] + """
>AND1
""" + AND1[1] + """
>OR1
""" + OR1[1] + """
@
"""

#Run RNAfold
p = subprocess.run(["RNAfold"], input=bytes(seq_all, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Collect RNAfold output for use in RNAplot
plot = p.stdout.decode() + "@"

#Check RNAfold output; optional
print(plot)

#Run RNAfold
p = subprocess.run(["RNAplot"], input=bytes(plot, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Check if there is error. Should output nothing here, but generate .ps files in the folder
print("***terminal message***")
print(p.stderr.decode())
print("***output file message***")
print(p.stdout.decode())


>YES1
GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
((((((((.(((((((((.......))))))))).((.((.(((...))).))))..(((((....))))).)))))))) (-33.00)
>NOT1
GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUGCGUCCUGUAUUCCACUGC
.((((....((((((.......((((......))))...((((((.((((..(((......)))..)))).)))))).))))))....)))) (-28.10)
>AND1
GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAUA
((((((((((((((((((((...(((.....(((.(((.......))).))).....)))..))))))).))))).....(((((....))))).))))))))......... (-42.10)
>OR1
GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
((((((((((((((((((((((.(((.....(((.(((.......))).))).)))...)).))))))).))))).....(((((....))))).)))))))) (-40.00)
@
***terminal message***

***output file message***



In [241]:
#Convert .ps files to .png files, so Markdown can read
p = subprocess.run(["convert","YES1_ss.ps","YES1_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)
p = subprocess.run(["convert","NOT1_ss.ps","NOT1_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)
p = subprocess.run(["convert","AND1_ss.ps","AND1_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)
p = subprocess.run(["convert","OR1_ss.ps","OR1_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

**RNA Secondary Structures for all riboswitches without contraints**

**YES-1** (below) Extra branch compared to paper
![YES-1](YES1_ss.png) 
**NOT-1** (below) Very similar structure to paper
![NOT-1](NOT1_ss.png) 
**AND-1** (below) Very similar structure to paper
![AND-1](AND1_ss.png) 
**OR-1** (below) Very similar structure to paper
![OR-1](OR1_ss.png) 

The default temperature is 37 DEG C, which is the temperature the authors used. The difference in stem-loop sturcture may be the different algorithms used. 

**Self-cleavage at binding sites?**

In [242]:
YES1_bindings = plot.split('\n')[2]
NOT1_bindings = plot.split('\n')[5]
AND1_bindings = plot.split('\n')[8]
OR1_bindings = plot.split('\n')[11]

#Check if self cleavage
def is_cleaved(bindings, data):
    if bindings[data[2]-1:data[3]].count('(') == len(bindings[data[2]-1:data[3]]) and bindings[data[4]-1:data[5]].count(')') == len(bindings[data[4]-1:data[5]]):
        if bindings[data[3]:data[4]-1].count('(') == bindings[data[3]:data[4]-1].count(')'):
            output = True
        else:
            output = False
    else:
        output = False
    return output

print("YES1 self cleavage at (red) binding sites?")
print(is_cleaved(YES1_bindings, YES1))
print("NOT1 self cleavage at (red) binding sites?")
print(is_cleaved(NOT1_bindings, NOT1))
print("AND1 self cleavage at (red) binding sites?")
print(is_cleaved(AND1_bindings, AND1))
print("OR1 self cleavage at (red) binding sites?")
print(is_cleaved(OR1_bindings, OR1))

YES1 self cleavage at (red) binding sites?
False
NOT1 self cleavage at (red) binding sites?
True
AND1 self cleavage at (red) binding sites?
False
OR1 self cleavage at (red) binding sites?
False


In [243]:
#Create YES-1_contraints text file

open('YES-1_contraints.txt', 'w+').close()

f = open('YES-1_contraints.txt', "a+")

for i in range(0, YES1[6]-1, 1):
    f.write('.')
for i in range (YES1[6]-1, YES1[7], 1):
    f.write('x')
for i in range (YES1[7], len(YES1[1]), 1):
    f.write('.')

f.close()

#Create NOT-1_contraints text file

open('NOT-1_contraints.txt', 'w+').close()

f = open('NOT-1_contraints.txt', "a+")

for i in range(0, NOT1[6]-1, 1):
    f.write('.')
for i in range (NOT1[6]-1, NOT1[7], 1):
    f.write('x')
for i in range (NOT1[7], len(NOT1[1]), 1):
    f.write('.')

f.close()


In [244]:
# #To Be Solved

# #Input YES-1 riboswitch sequences
# seq_YES1 = """>YES1
# """ + YES1[1] + """
# @
# """

# #Run RNAfold - THIS STEP HAS ISSUES, "-C" command does not work in Terminal either.
# p = subprocess.run(["RNAfold","-C","YES-1_contraints.txt"], input=bytes(seq_YES1, 'ascii'),
#                       stdout=subprocess.PIPE, stderr=subprocess.PIPE,
#                       check=True)

# #Collect RNAfold output for use in RNAplot
# plot = p.stdout.decode() + "@"

# #Check RNAfold output; optional
# print(plot)

# #Run RNAfold
# p = subprocess.run(["RNAplot"], input=bytes(plot, 'ascii'),
#                       stdout=subprocess.PIPE, stderr=subprocess.PIPE,
#                       check=True)

# #Check if there is error. Should output nothing here, but generate .ps files in the folder
# print("***terminal message***")
# print(p.stderr.decode())
# print("***output file message***")
# print(p.stdout.decode())


For RNAfold and plot of YES-1 with contraints, slightly different method from instruction. Running "RNAfold -C YES-1_contraints.txt did not work. Error message was unable to read YES-1_contraints.txt. Alternately, ran"RNfAfold -C YES_contrains" as argument and made the contents of YES-1_contraints.txt part of the input.

In [245]:
#Input YES-1 riboswitch sequences and contraints
seq_YES1 = """>YES1_with_contraints
""" + YES1[1] + """
""" + open("YES-1_contraints.txt", "r").read() + """
""" + """
@
"""

#Run RNAplot
p = subprocess.run(["RNAfold","-C"], input=bytes(seq_YES1, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Collect RNAfold output for use in RNAplot
plot = p.stdout.decode() + "@"

#Check RNAfold output; optional
print(plot)

#Check if self cleavage
YES1_bindings = plot.split('\n')[2]

print("\nYES1 self cleavage at (red) binding sites?")
print(is_cleaved(YES1_bindings, YES1))
print("\n")

#Run RNAfold
p = subprocess.run(["RNAplot"], input=bytes(plot, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Check if there is error. Should output nothing here, but generate .ps files in the folder
print("***terminal message***")
print(p.stderr.decode())
print("***output file message***")
print(p.stdout.decode())

#Convert .ps files to .png files, so Markdown can read
p = subprocess.run(["convert","YES1_with_contraints_ss.ps","YES1_with_contraints_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

>YES1_with_contraints
GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
((((((((.......((((((...........................))))))...(((((....))))).)))))))) (-24.50)
@

YES1 self cleavage at (red) binding sites?
True


***terminal message***

***output file message***



In [246]:
#RNAfold and plot for NOT-1 with contraints

#Input NOT-1 riboswitch sequences and contraints
seq_NOT1 = """>NOT1_with_contraints
""" + NOT1[1] + """
""" + open("NOT-1_contraints.txt", "r").read() + """
""" + """
@
"""

#Run RNAplot
p = subprocess.run(["RNAfold","-C"], input=bytes(seq_NOT1, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Collect RNAfold output for use in RNAplot
plot = p.stdout.decode() + "@"

#Check RNAfold output; optional
print(plot)

#Check if self cleavage
NOT1_bindings = plot.split('\n')[2]

print("\nNOT1 self cleavage at (red) binding sites?")
print(is_cleaved(NOT1_bindings, NOT1))
print("\n")

#Run RNAfold
p = subprocess.run(["RNAplot"], input=bytes(plot, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Check if there is error. Should output nothing here, but generate .ps files in the folder
print("***terminal message***")
print(p.stderr.decode())
print("***output file message***")
print(p.stdout.decode())

#Convert .ps files to .png files, so Markdown can read
p = subprocess.run(["convert","NOT1_with_contraints_ss.ps","NOT1_with_contraints_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

>NOT1_with_contraints
GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUGCGUCCUGUAUUCCACUGC
.((((....((((((..((((.((((......))))(((((((.......................))))))).))))))))))....)))) (-20.80)
@

NOT1 self cleavage at (red) binding sites?
False


***terminal message***

***output file message***



**RNA Secondary Structures for YES-1 and NOT-1 riboswitches with contraints**

![YES-1 with contraints](YES1_with_contraints_ss.png) 
![NOT-1 with contraints](NOT1_with_contraints_ss.png) 

In both YES-1 (above) and NOT-1 (below), the folded sturctures look exactly as the paper. In YES-1, the red regions are bound to each other and self-cleavage occurs. In NOT-1, the red regions are not bound to each other, no self-cleavage occurs.

In [247]:
#Create AND1_OBS1_contraints text file

open('AND-1_OBS1_contraints.txt', 'w+').close()

f = open('AND-1_OBS1_contraints.txt', "a+")

for i in range(0, AND1[6]-1, 1):
    f.write('.')
for i in range (AND1[6]-1, AND1[7], 1):
    f.write('x')
for i in range (AND1[7], len(AND1[1]), 1):
    f.write('.')

f.close()

#Create AND-1_OBS2_contraints text file

open('AND-1_OBS2_contraints.txt', 'w+').close()

f = open('AND-1_OBS2_contraints.txt', "a+")

for i in range(0, AND1[8]-1, 1):
    f.write('.')
for i in range (AND1[8]-1, AND1[9], 1):
    f.write('x')
for i in range (AND1[9], len(AND1[1]), 1):
    f.write('.')

f.close()

#Create AND-1_OBS1_OBS2_contraints text file

open('AND-1_OBS1_OBS2_contraints.txt', 'w+').close()

f = open('AND-1_OBS1_OBS2_contraints.txt', "a+")

for i in range(0, AND1[6]-1, 1):
    f.write('.')
for i in range (AND1[6]-1, AND1[7], 1):
    f.write('x')
for i in range (AND1[7], AND1[8]-1, 1):
    f.write('.')
for i in range (AND1[8]-1, AND1[9], 1):
    f.write('x')
for i in range (AND1[9], len(AND1[1]), 1):
    f.write('.')

f.close()

In [248]:
#RNAfold and plot for AND-1 with OBS1 binding only

#Input AND-1 riboswitch sequences and contraints, OBS1 binding only
seq_AND1_OBS1 = """>AND1_OBS1_with_contraints
""" + AND1[1] + """
""" + open("AND-1_OBS1_contraints.txt", "r").read() + """
""" + """
@
"""

#Run RNAplot
p = subprocess.run(["RNAfold","-C"], input=bytes(seq_AND1_OBS1, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Collect RNAfold output for use in RNAplot
plot = p.stdout.decode() + "@"

#Check RNAfold output; optional
print(plot)

#Check if self-cleavage in AND-1 OBS1 binding condition
AND1_OBS1_bindings = plot.split('\n')[2]

print("\nAND1_OBS1 self cleavage at (red) binding sites?")
print(is_cleaved(AND1_OBS1_bindings, AND1))
print("\n")

#Run RNAfold
p = subprocess.run(["RNAplot"], input=bytes(plot, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Check if there is error. Should output nothing here, but generate .ps files in the folder
print("***terminal message***")
print(p.stderr.decode())
print("***output file message***")
print(p.stdout.decode())

>AND1_OBS1_with_contraints
GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAUA
((((((((((((((((((((..........................(((.(....))))...))))))).))))).....(((((....))))).))))))))......... (-33.90)
@

AND1_OBS1 self cleavage at (red) binding sites?
False


***terminal message***

***output file message***



In [249]:
#RNAfold and plot for AND-1 with OBS2 binding only

#Input AND-1 riboswitch sequences and contraints, OBS2 binding only
seq_AND1_OBS2 = """>AND1_OBS2_with_contraints
""" + AND1[1] + """
""" + open("AND-1_OBS2_contraints.txt", "r").read() + """
""" + """
@
"""

#Run RNAplot
p = subprocess.run(["RNAfold","-C"], input=bytes(seq_AND1_OBS2, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Collect RNAfold output for use in RNAplot
plot = p.stdout.decode() + "@"

#Check RNAfold output; optional
print(plot)

#Check if self-cleavage in AND-1 OBS2 binding condition
AND1_OBS2_bindings = plot.split('\n')[2]

print("\nAND1_OBS2 self cleavage at (red) binding sites?")
print(is_cleaved(AND1_OBS2_bindings, AND1))
print("\n")

#Run RNAfold
p = subprocess.run(["RNAplot"], input=bytes(plot, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Check if there is error. Should output nothing here, but generate .ps files in the folder
print("***terminal message***")
print(p.stderr.decode())
print("***output file message***")
print(p.stdout.decode())

>AND1_OBS2_with_contraints
GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAUA
(((((((((((((((((((((...(((.......))))))........................))))).))))).....(((((....))))).))))))))......... (-29.60)
@

AND1_OBS2 self cleavage at (red) binding sites?
False


***terminal message***

***output file message***



In [250]:
#RNAfold and plot for AND-1 with OBS1 and OBS2 binding

#Input AND-1 riboswitch sequences and contraints, both OBS1 and OBS2 binding
seq_AND1_OBS1_OBS2 = """>AND1_OBS1_OBS2_with_contraints
""" + AND1[1] + """
""" + open("AND-1_OBS1_OBS2_contraints.txt", "r").read() + """
""" + """
@
"""

#Run RNAplot
p = subprocess.run(["RNAfold","-C"], input=bytes(seq_AND1_OBS1_OBS2, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Collect RNAfold output for use in RNAplot
plot = p.stdout.decode() + "@"

#Check RNAfold output; optional
print(plot)

#Check if self-cleavage in AND-1 OBS1 and OBS2 binding condition
AND1_OBS1_OBS2_bindings = plot.split('\n')[2]

print("\nAND1_OBS1_OBS2 self cleavage at (red) binding sites?")
print(is_cleaved(AND1_OBS1_OBS2_bindings, AND1))
print("\n")

#Run RNAfold
p = subprocess.run(["RNAplot"], input=bytes(plot, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Check if there is error. Should output nothing here, but generate .ps files in the folder
print("***terminal message***")
print(p.stderr.decode())
print("***output file message***")
print(p.stdout.decode())

>AND1_OBS1_OBS2_with_contraints
GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAUA
((((((((((((((((((..............................................))))).))))).....(((((....))))).))))))))......... (-27.64)
@

AND1_OBS1_OBS2 self cleavage at (red) binding sites?
False


***terminal message***

***output file message***



In [251]:
#Convert .ps files to .png files, so Markdown can read
p = subprocess.run(["convert","AND1_OBS1_with_contraints_ss.ps","AND1_OBS1_with_contraints_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)
p = subprocess.run(["convert","AND1_OBS2_with_contraints_ss.ps","AND1_OBS2_with_contraints_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)
p = subprocess.run(["convert","AND1_OBS1_OBS2_with_contraints_ss.ps","AND1_OBS1_OBS2_with_contraints_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

**RNA Secondary Structures for AND1 riboswitches with contraints**

**AND1_OBS1_with_contraints** (below) Extra branch when compared to paper
![AND1_OBS1_with_contraints](AND1_OBS1_with_contraints_ss.png) 
**AND1_OBS2_with_contraints** (below) Very similar structure to paper
![AND1_OBS2_with_contraints](AND1_OBS2_with_contraints_ss.png) 
**AND1_OBS1_OBS2_with_contraints** (below) Very similar structure to paper
![AND1_OBS1_OBS2_with_contraints](AND1_OBS1_OBS2_with_contraints_ss.png) 


**Self-cleavage at binding sites?**

In [252]:
print("AND1_OBS1 self cleavage at (red) binding sites?")
print(is_cleaved(AND1_OBS1_bindings, AND1))
print("\n")
print("AND1_OBS2 self cleavage at (red) binding sites?")
print(is_cleaved(AND1_OBS2_bindings, AND1))
print("\n")
print("AND1_OBS1_OBS2 self cleavage at (red) binding sites?")
print(is_cleaved(AND1_OBS1_OBS2_bindings, AND1))
print("\n")

AND1_OBS1 self cleavage at (red) binding sites?
False


AND1_OBS2 self cleavage at (red) binding sites?
False


AND1_OBS1_OBS2 self cleavage at (red) binding sites?
False




In [253]:
#Create OR1_OBS1_contraints text file

open('OR-1_OBS1_contraints.txt', 'w+').close()

f = open('OR-1_OBS1_contraints.txt', "a+")

for i in range(0, OR1[6]-1, 1):
    f.write('.')
for i in range (OR1[6]-1, OR1[7], 1):
    f.write('x')
for i in range (OR1[7], len(OR1[1]), 1):
    f.write('.')

f.close()

#Create OR-1_OBS2_contraints text file

open('OR-1_OBS2_contraints.txt', 'w+').close()

f = open('OR-1_OBS2_contraints.txt', "a+")

for i in range(0, OR1[8]-1, 1):
    f.write('.')
for i in range (OR1[8]-1, OR1[9], 1):
    f.write('x')
for i in range (OR1[9], len(OR1[1]), 1):
    f.write('.')

f.close()

#Create OR-1_OBS1_OBS2_contraints text file

open('OR-1_OBS1_OBS2_contraints.txt', 'w+').close()

f = open('OR-1_OBS1_OBS2_contraints.txt', "a+")

for i in range(0, OR1[6]-1, 1):
    f.write('.')
for i in range (OR1[6]-1, OR1[7], 1):
    f.write('x')
for i in range (OR1[7], OR1[8]-1, 1):
    f.write('.')
for i in range (OR1[8]-1, OR1[9], 1):
    f.write('x')
for i in range (OR1[9], len(OR1[1]), 1):
    f.write('.')

f.close()

In [254]:
#RNAfold and plot for OR-1 with OBS1 binding only

#Input OR-1 riboswitch sequences and contraints, OBS1 binding only
seq_OR1_OBS1 = """>OR1_OBS1_with_contraints
""" + OR1[1] + """
""" + open("OR-1_OBS1_contraints.txt", "r").read() + """
""" + """
@
"""

#Run RNAplot
p = subprocess.run(["RNAfold","-C"], input=bytes(seq_OR1_OBS1, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Collect RNAfold output for use in RNAplot
plot = p.stdout.decode() + "@"

#Check RNAfold output; optional
print(plot)

#Check if self-cleavage in OR-1 OBS1 binding condition
OR1_OBS1_bindings = plot.split('\n')[2]

print("\nOR1_OBS1 self cleavage at (red) binding sites?")
print(is_cleaved(OR1_OBS1_bindings, OR1))
print("\n")

#Run RNAfold
p = subprocess.run(["RNAplot"], input=bytes(plot, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Check if there is error. Should output nothing here, but generate .ps files in the folder
print("***terminal message***")
print(p.stderr.decode())
print("***output file message***")
print(p.stdout.decode())

>OR1_OBS1_with_contraints
GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
((((((((((((((((((((((.(((...........................)))...)).))))))).))))).....(((((....))))).)))))))) (-34.20)
@

OR1_OBS1 self cleavage at (red) binding sites?
False


***terminal message***

***output file message***



In [255]:
#RNAfold and plot for OR-1 with OBS2 binding only

#Input OR-1 riboswitch sequences and contraints, OBS2 binding only
seq_OR1_OBS2 = """>OR1_OBS2_with_contraints
""" + OR1[1] + """
""" + open("OR-1_OBS2_contraints.txt", "r").read() + """
""" + """
@
"""

#Run RNAplot
p = subprocess.run(["RNAfold","-C"], input=bytes(seq_OR1_OBS2, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Collect RNAfold output for use in RNAplot
plot = p.stdout.decode() + "@"

#Check RNAfold output; optional
print(plot)

#Check if self-cleavage in OR-1 OBS2 binding condition
OR1_OBS2_bindings = plot.split('\n')[2]

print("\nOR1_OBS2 self cleavage at (red) binding sites?")
print(is_cleaved(OR1_OBS2_bindings, OR1))
print("\n")

#Run RNAfold
p = subprocess.run(["RNAplot"], input=bytes(plot, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Check if there is error. Should output nothing here, but generate .ps files in the folder
print("***terminal message***")
print(p.stderr.decode())
print("***output file message***")
print(p.stdout.decode())

>OR1_OBS2_with_contraints
GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
((((((((.......(((((((((((........................................)))))))))))...(((((....))))).)))))))) (-29.09)
@

OR1_OBS2 self cleavage at (red) binding sites?
True


***terminal message***

***output file message***



In [256]:
#RNAfold and plot for OR-1 with OBS1 and OBS2 binding

#Input OR-1 riboswitch sequences and contraints, both OBS1 and OBS2 binding
seq_OR1_OBS1_OBS2 = """>OR1_OBS1_OBS2_with_contraints
""" + OR1[1] + """
""" + open("OR-1_OBS1_OBS2_contraints.txt", "r").read() + """
""" + """
@
"""

#Run RNAplot
p = subprocess.run(["RNAfold","-C"], input=bytes(seq_OR1_OBS1_OBS2, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Collect RNAfold output for use in RNAplot
plot = p.stdout.decode() + "@"

#Check RNAfold output; optional
print(plot)

#Check if self-cleavage in AND-1 OBS1 and OBS2 binding condition
OR1_OBS1_OBS2_bindings = plot.split('\n')[2]

print("\nOR1_OBS1_OBS2 self cleavage at (red) binding sites?")
print(is_cleaved(OR1_OBS1_OBS2_bindings, OR1))
print("\n")

#Run RNAfold
p = subprocess.run(["RNAplot"], input=bytes(plot, 'ascii'),
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

#Check if there is error. Should output nothing here, but generate .ps files in the folder
print("***terminal message***")
print(p.stderr.decode())
print("***output file message***")
print(p.stdout.decode())

>OR1_OBS1_OBS2_with_contraints
GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
((((((((.......(((((((((((........................................)))))))))))...(((((....))))).)))))))) (-29.09)
@

OR1_OBS1_OBS2 self cleavage at (red) binding sites?
True


***terminal message***

***output file message***



In [257]:
#Convert .ps files to .png files, so Markdown can read
p = subprocess.run(["convert","OR1_OBS1_with_contraints_ss.ps","OR1_OBS1_with_contraints_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)
p = subprocess.run(["convert","OR1_OBS2_with_contraints_ss.ps","OR1_OBS2_with_contraints_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)
p = subprocess.run(["convert","OR1_OBS1_OBS2_with_contraints_ss.ps","OR1_OBS1_OBS2_with_contraints_ss.png"],
                      stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                      check=True)

**RNA Secondary Structures for OR1 riboswitches with contraints**

**OR1_OBS1_with_contraints** (below) Very similar structure to paper
![OR1_OBS1_with_contraints](OR1_OBS1_with_contraints_ss.png) 
**OR1_OBS2_with_contraints** (below) Very similar structure to paper
![OR1_OBS2_with_contraints](OR1_OBS2_with_contraints_ss.png) 
**OR1_OBS1_OBS2_with_contraints** (below) Very similar structure to paper
![OR1_OBS1_OBS2_with_contraints](OR1_OBS1_OBS2_with_contraints_ss.png) 


**Self-cleavage at binding sites?**

In [258]:
print("OR1_OBS1 self cleavage at (red) binding sites?")
print(is_cleaved(OR1_OBS1_bindings, OR1))
print("\n")
print("OR1_OBS2 self cleavage at (red) binding sites?")
print(is_cleaved(OR1_OBS2_bindings, OR1))
print("\n")
print("OR1_OBS1_OBS2 self cleavage at (red) binding sites?")
print(is_cleaved(OR1_OBS1_OBS2_bindings, OR1))
print("\n")

OR1_OBS1 self cleavage at (red) binding sites?
False


OR1_OBS2 self cleavage at (red) binding sites?
True


OR1_OBS1_OBS2 self cleavage at (red) binding sites?
True




In [259]:
#Generate Truth table for AND-1

c.execute("""DROP TABLE AND1_truth_table;""")
 
c.execute("""CREATE TABLE AND1_truth_table 
(OBS1_binding TEXT, 
OBS2_binding TEXT, 
Self_cleavage TEXT);""")

#input data for AND-1 Truth table
AND1_truth_table_data = [('True', 'True', str(is_cleaved(AND1_OBS1_OBS2_bindings, AND1))),
                   ('True', 'False', str(is_cleaved(AND1_OBS1_bindings, AND1))),
                   ('False', 'True', str(is_cleaved(AND1_OBS2_bindings, AND1))),
                   ('False', 'False', str(is_cleaved(AND1_bindings, AND1)))
                  ]

c.executemany('INSERT INTO AND1_truth_table VALUES (?,?,?)', AND1_truth_table_data)

conn.commit()

c.execute("SELECT * FROM AND1_truth_table;")
print(c.fetchall())

[('True', 'True', 'False'), ('True', 'False', 'False'), ('False', 'True', 'False'), ('False', 'False', 'False')]


In [260]:
#Generate Truth table for OR-1

c.execute("""DROP TABLE OR1_truth_table;""")
 
c.execute("""CREATE TABLE OR1_truth_table 
(OBS1_binding TEXT, 
OBS2_binding TEXT, 
Self_cleavage TEXT);""")

#input data for OR-1 Truth table
OR1_truth_table_data = [('True', 'True', str(is_cleaved(OR1_OBS1_OBS2_bindings, OR1))),
                   ('True', 'False', str(is_cleaved(OR1_OBS1_bindings, OR1))),
                   ('False', 'True', str(is_cleaved(OR1_OBS2_bindings, OR1))),
                   ('False', 'False', str(is_cleaved(OR1_bindings, OR1)))
                  ]

c.executemany('INSERT INTO OR1_truth_table VALUES (?,?,?)', OR1_truth_table_data)

conn.commit()

c.execute("SELECT * FROM OR1_truth_table;")
print(c.fetchall())

[('True', 'True', 'True'), ('True', 'False', 'False'), ('False', 'True', 'True'), ('False', 'False', 'False')]


***According to our results, AND-1 and OR-1 riboswitches do not work as the paper claim. RNAfold does not generate the same folding structure as the ones in the paper, so the relative locations of the red regions are different from the paper.***  
If we had access to more information about how they generated their models, we could more accurately be able to replicate their results. Our binding site likely match but not exactly (1:1 base binding) 